In [1]:
#this generator uses generator_dotproduct functions.  
import math
import os
import sys
import import_ipynb
from Generator_DotProduct_N_SI_HR_v4_6_AXI_Stream_function import DotProduct_SI_AXI_Stream 
from Generator_DotProduct_noHR_AXIStream import DotProuct_noHR_AXIStream

importing Jupyter notebook from Generator_DotProduct_N_SI_HR_v4_6_AXI_Stream_function.ipynb
importing Jupyter notebook from Generator_DotProduct_N_SI_HR_v4_6.ipynb
importing Jupyter notebook from DotProduct_SoE_V4_5.ipynb
importing Jupyter notebook from DotProduct_Systolic_V1_11.ipynb
importing Jupyter notebook from DotProduct_noHR_M_V3_7.ipynb
importing Jupyter notebook from RegisteredNto1Adder_V1_1.ipynb
importing Jupyter notebook from DotProduct_noHR_C_V3_7.ipynb
importing Jupyter notebook from AdderTree_ASm_V1_20.ipynb
importing Jupyter notebook from SItoSoE_V1_10.ipynb
HRR calculated value: 2
HRR calculated value: 2
importing Jupyter notebook from Generator_DotProduct_noHR_AXIStream.ipynb


In [2]:
#n is current sub determinant dimension.
#N is total determinant dimensiion.
#RowMask is zero for omitted rows in sub determinant
#columnMask is zero for omitted columns in sub determinant
#p is row index of previous element which made this sub determinant
#k is column index of previous element which made this sub determinant
#dpInstanceCount is a list for dot products count in rach level of tree. and from index 2 to N values are count of defined dps for fp levels 2 to N.
#dpNamesDic is a dictionary that have names of dp modules of different lenght in it.
#fileObject is for printing generating verilog code.
#outDataWireName specifies the output wire of current sub determinant Data in interconnect network.
#outreadyWireName specifies the output wire of current sub determinant Ready signal in interconnect network.
#outVaidWireName specifies the output wire of current sub determinant Valid signal in interconnect network.
#dp2sReadyWire dpNReadyWire anded together connect to s_axi_ready of determinant top module.
#dpIndataLenghtDic specifies each dp layers input and output width.
#dpOutdataLenghtDic specifies each dp layers input and output width.
#dpInwitdhDic specifies each dp single element width in bits.
#determinantElementWidth specifies NxN determinant elements width.
#determinantIndataLenght specifies NxN determinant elements width.
#dpInstancesDic specifies the dp instances with specific rowmask and collumn mask that are already created.
#preventDuplicateDps is a flag for preventing redundant dps creation. 
def determinant_NXN(N,n,RowMask,CollumnMask,p,k,dpInwidthDic,dpNamesDic,dpIndataLenghtDic,dpOutdataLenghtDic\
                    ,dpInstanceCount,fileObject,outDataWireName,outReadyWireName,outValidWireName,dp2sReadyWire\
                    ,dpNReadyWire,determinantElementWidth,determinantIndataLenght,dpInstancesDic,preventDuplicateDps):
    
    RowCollumnMaskKey=(''.join(map(str,RowMask)))+(''.join(map(str,CollumnMask)))
    if((RowCollumnMaskKey in dpInstancesDic) and (preventDuplicateDps==1)):
        print("assign {}={};".format(outDataWireName,dpInstancesDic[RowCollumnMaskKey][0]),file=fileObject)
        #print("assign {}={};".format(outReadyWireName,dpInstancesDic[RowCollumnMaskKey][1]),file=fileObject)
        print("assign {}={};".format(outValidWireName,dpInstancesDic[RowCollumnMaskKey][2]),file=fileObject)                                         
    else:
        dpInstancesDic[RowCollumnMaskKey]=[outDataWireName,outReadyWireName,outValidWireName]
        if(n==2):
            #create a 2_elements dotProduct
            print("{} dp{}_{}(".format(dpNamesDic["dp{}".format(n)],n,dpInstanceCount[n]),file=fileObject)
            print(".aclk(aclk),",file=fileObject) 
            print(".aresetn(aresetn),",file=fileObject) 
            print(".enable(enable),",file=fileObject)
            print(".s_axi_data({",file=fileObject)  
            firstColumnFlag=1
            for i in range(N):
                if(CollumnMask[i]==1):
                    startIndex1=determinantIndataLenght-(N*(N-2)+i)*determinantElementWidth-1
                    endIndex1=determinantIndataLenght-(N*(N-2)+i+1)*determinantElementWidth
                    startIndex2=determinantIndataLenght-(N*(N-2)+i+N)*determinantElementWidth-1
                    endIndex2=determinantIndataLenght-(N*(N-2)+i+N+1)*determinantElementWidth
                    if(firstColumnFlag==1):
                        print("s_axi_data[{}:{}],".format(startIndex1,endIndex1),file=fileObject)
                        print("s_axi_data[{}:{}],".format(startIndex2,endIndex2),file=fileObject)
                        firstColumnFlag=0
                    else: 
                        print("s_axi_data[{}:{}],".format(startIndex2,endIndex2),file=fileObject)
                        print("-s_axi_data[{}:{}]".format(startIndex1,endIndex1),file=fileObject)
            print("}),",file=fileObject)
            if(dpInstanceCount[n]==0):# we connect just one of dp2 inready signal to dp2sReadyWire
                print(".s_axi_ready({}),".format(dp2sReadyWire),file=fileObject)
            print(".s_axi_valid(s_axi_valid),",file=fileObject)
            print(".m_axi_data({}),".format(outDataWireName),file=fileObject)
            print(".m_axi_ready({}),".format(outReadyWireName),file=fileObject)
            print(".m_axi_valid({})".format(outValidWireName),file=fileObject)
            print(");",file=fileObject)
            dpInstanceCount[n]+=1
            return
        else:
            #create an n_elements dotproduct.
            print("wire dp{}_{}_s_axi_ready;".format(n,dpInstanceCount[n]),file=fileObject)  
            if(n==N):
                print("assign {}=dp{}_{}_s_axi_ready;".format(dpNReadyWire,n,dpInstanceCount[n]),file=fileObject)
            print("wire dp{}_{}_s_axi_valid;".format(n,dpInstanceCount[n]),file=fileObject)
            print("wire signed [{}:{}]dp{}_{}_s_axi_data;".format(dpIndataLenghtDic["dp{}".format(n)]-1,0,n,dpInstanceCount[n]),file=fileObject)              
            for i in range(n):
                print("wire signed [{}:0]dp{}_{}element{};".format(dpInwidthDic["dp{}".format(n)]-1,n,dpInstanceCount[n],i),file=fileObject)
                print("wire dp{}_{}valid{};".format(n,dpInstanceCount[n],i),file=fileObject)
            print("assign dp{}_{}_s_axi_valid=".format(n,dpInstanceCount[n]),file=fileObject,end='')
            for i in range(n):
                print("dp{}_{}valid{}".format(n,dpInstanceCount[n],i) ,file=fileObject,end='')
                if(i!=n-1):
                    print("&&",file=fileObject,end='')
            print(";",file=fileObject)
            print("assign dp{}_{}_s_axi_data={{".format(n,dpInstanceCount[n]),file=fileObject,end='')              
            for i in range(n):#this loop connects results comming from lower dimension dotproducts to current dotproduct input.    
                print("dp{}_{}element{},".format(n,dpInstanceCount[n],i),file=fileObject,end='')
            coefficientSign=1
            elementsNumber=0   #this  variable counts the n elemnts of coeficient for current dotproduct.
            for i in range (N):   #this loop connects coefficent elements of current dotproduct.
                if(CollumnMask[i]==1):
                    elementsNumber+=1
                    startIndex=determinantIndataLenght-(N*(N-n)+i)*determinantElementWidth-1
                    endIndex=determinantIndataLenght-(N*(N-n)+i+1)*determinantElementWidth
                    if(coefficientSign==1):
                        print("{{{{{}{{coefficientsShiftRegister_{}[{}]}}}},{{coefficientsShiftRegister_{}[{}:{}]}}}}".format(dpInwidthDic["dp{}".format(n)]-determinantElementWidth,n,startIndex,n,startIndex,endIndex),file=fileObject,end='')  
                    else:
                        print("-{{{{{}{{coefficientsShiftRegister_{}[{}]}}}},{{coefficientsShiftRegister_{}[{}:{}]}}}}".format(dpInwidthDic["dp{}".format(n)]-determinantElementWidth,n,startIndex,n,startIndex,endIndex),file=fileObject,end='')
                    if(elementsNumber!=n):
                        print(",",file=fileObject)  
                    coefficientSign*=-1
            print("};",file=fileObject)
            print("{} dp{}_{}(".format(dpNamesDic["dp{}".format(n)],n,dpInstanceCount[n]),file=fileObject)
            print(".aclk(aclk),",file=fileObject)
            print(".aresetn(aresetn),",file=fileObject)
            print(".enable(enable),",file=fileObject)
            print(".s_axi_data(dp{}_{}_s_axi_data),".format(n,dpInstanceCount[n]),file=fileObject)
            print(".s_axi_ready(dp{}_{}_s_axi_ready),".format(n,dpInstanceCount[n]),file=fileObject)
            print(".s_axi_valid(dp{}_{}_s_axi_valid),".format(n,dpInstanceCount[n]),file=fileObject)
            print(".m_axi_data({}),".format(outDataWireName),file=fileObject)
            print(".m_axi_ready({}),".format(outReadyWireName),file=fileObject)
            print(".m_axi_valid({})".format(outValidWireName),file=fileObject)
            print(");",file=fileObject)                  
            elementsNumber=0
            for i in range(N):
                if(CollumnMask[i]==1):
                    outDataWireName="dp{}_{}element{}".format(n,dpInstanceCount[n],elementsNumber)
                    outReadyWireName="dp{}_{}_s_axi_ready".format(n,dpInstanceCount[n])
                    outValidWireName="dp{}_{}valid{}".format(n,dpInstanceCount[n],elementsNumber)
                    CollumnMask[i]=0
                    RowMask[i]=0
                    determinant_NXN(N,n-1,RowMask,CollumnMask,N-n,i,dpInwidthDic,dpNamesDic,dpIndataLenghtDic,dpOutdataLenghtDic\
                                    ,dpInstanceCount,fileObject,outDataWireName,outReadyWireName,outValidWireName,dp2sReadyWire\
                                    ,dpNReadyWire,determinantElementWidth,determinantIndataLenght,dpInstancesDic,preventDuplicateDps)          
                    elementsNumber+=1
                    #create subdeterminant
                    CollumnMask[i]=1
                    RowMask[i]=1
            dpInstanceCount[n]+=1
            #RowMask[p]=0
            #CollumnMask[k]=0
            return

In [ ]:
N=input('enter N, square determinant dimension (empty for 2): ')
if not N:
    N=2
else:
    N= int(N) 
IN_WIDTH=input('enter width of each element of determinant in bits (empty for 10): ')
if not IN_WIDTH:
     IN_WIDTH = 10
else:
    IN_WIDTH = int(IN_WIDTH)
#hardware reuse value will reduce for sub determinants(dotproducts).
HRR=input('enter hardware reuse rate(2>= , empty or 1 for no HardwareReuse):')
HRFlag=False
if not HRR:
    HRFlag=False
else:   
    HRR = int(HRR)
    if HRR==1:
        HRFlag=False
    if (HRR<2 and HRR>1):
        HRR = 2
        HRFlag=True
    if HRR>=2:
        HRFlag=True 
        if(HRR>=N):
            HRR=N
preventDuplicateDps=input('enter preventDuplicateDps(empty or 1 for true and 0 for false): ')
if not preventDuplicateDps:
    preventDuplicateDps = 1
else:
    preventDuplicateDps = int(preventDuplicateDps)
    if(preventDuplicateDps>=1):
        preventDuplicateDps=1
    else:
        preventDuplicateDps=0
    
#creation and filling the following four dictionaries:
dpInstancesDic={}
dpInwidthDic={}
dpIndataLenghtDic={}
dpOutdataLenghtDic={}
for i in range(2,N+1):
    if(i==2):
        dpInwidthDic["dp{}".format(i)]=IN_WIDTH
    else:
        multiplicationExtend=2*dpInwidthDic["dp{}".format(i-1)]
        summationExtend=math.ceil(math.log2(i-1))
        dpInwidthDic["dp{}".format(i)]=multiplicationExtend+summationExtend
    dpIndataLenghtDic["dp{}".format(i)]=2*i*dpInwidthDic["dp{}".format(i)]
    
    multiplicationExtend=2*dpInwidthDic["dp{}".format(i)]
    summationExtend=math.ceil(math.log2(i))
    dpOutdataLenghtDic["dp{}".format(i)]=multiplicationExtend+summationExtend
#filling other input argument for determinant function:
RowMask=[1]*N
CollumnMask=[1]*N
dpInstanceCount=[0]*(N+1) #initially a list of zeros. 
determinantElementWidth=IN_WIDTH
determinantIndataLenght=N*N*IN_WIDTH
originalHRR=HRR
dpNamesDic={}
parameters={}
modules={}

Files_Location="Determinant_GeneratedVerilogCodes/"
Print_To_File=1
Is_Top=1
ModuleName='determinant'
ModuleName+='_{}x{}_{}bit_'.format(N,N,IN_WIDTH)
if(HRFlag):
    ModuleName+='_maxHRx{}_'.format(originalHRR)
else:
    ModuleName+='_noHR'
if(preventDuplicateDps):
    ModuleName+='_PDs'
else:
    ModuleName+='_NPDs'
ModuleName+='_AXIStream'
if Is_Top:
    Files_Location += ModuleName+"/"
    if not os.path.exists(Files_Location):
        os.makedirs(Files_Location)

if Print_To_File<=0:
    of = sys.stdout
else:
    of = open(Files_Location+ModuleName+".v", 'w+')
fileObject=of    
for i in range(2,N+1):
    print("creating determinant {}x{}".format(i,i))
    HRR=originalHRR
    if(HRFlag==True):#for Hardware Reuse activated conditions
        NMult = math.ceil(i/HRR)
        while math.ceil(i/NMult) != HRR:
            HRR -= 1
        print("HRR calculated value for dp{} is: {}".format(i,HRR))
        #MAMCS>=1
        MAMCS=input('enter MAMCS for dp{}:(empty for {})'.format(i,NMult))  
        if not MAMCS:
            MAMCS = NMult
        else:
            MAMCS = int(MAMCS)
            if MAMCS>NMult:
                MAMCS = NMult
            elif MAMCS<1:
                MAMCS = 1
        Enable_Input_Latch =input("enter ENABLE_INPUT_LATCH for dp{}(empty or zero for disable):".format(i))
        NumOfComponents = math.ceil(NMult/MAMCS)
        #AdderSize>=2
        print("Nmult for dp{}is:{}".format(i,NMult))
        if MAMCS==NMult:
            AdderSize = 0
        else:
            if MAMCS<NMult:
                AdderSize = input('Enter maximum size allowed for base adders (empty for 2): ')
                if not AdderSize:
                    AdderSize = 2
                else:
                    AdderSize = int(AdderSize)
                    if AdderSize<2:
                        AdderSize = 2
                    if AdderSize>NumOfComponents:
                        AdderSize = NumOfComponents
        if not Enable_Input_Latch:
            Enable_Input_Latch=0
        else:
            Enable_Input_Latch = int(Enable_Input_Latch)
        parameters={ 
            "N":i,
            "HRR":HRR,
            "MAMCS":MAMCS,
            "AdderSize":AdderSize,
            "IN_WIDTH":dpInwidthDic["dp{}".format(i)],
            "ENABLE_INPUT_LATCH":Enable_Input_Latch
        }
    else: #for HardwareReuse diactivated conditions.
        #MAMCS>=2
        MAMCS = input('Enter Multiply-Add Max Component Size (>=2, <= {}, empty for 2): '.format(N-1)) 
        #AdderSize>=2
        AdderSize = input('Enter maximum size allowed for base adders (empty for 2): ')
        #INPUT_REG_DEPTH>=0
        if (not MAMCS):
            MAMCS = N-1
        else:
            MAMCS = int(MAMCS)
        if MAMCS>=(N-1):
            MAMCS = N-1
            NumOfMaxSizeComponents=1
            NonMaxSizeComponentSize=1;
        else:
            if(MAMCS<2):
                MAMCS=2
            NumOfMaxSizeComponents = math.floor(N/MAMCS)
            if (N/MAMCS) == NumOfMaxSizeComponents:
                NonMaxSizeComponentSize=0
            else:
                NonMaxSizeComponentSize = N - MAMCS*NumOfMaxSizeComponents
        NumOfComponents = NumOfMaxSizeComponents + (NonMaxSizeComponentSize!=0)

        if not AdderSize:
            AdderSize=2
        else:
            AdderSize = int(AdderSize)
            if AdderSize<2:
                AdderSize=2
            if AdderSize>NumOfComponents:
                AdderSize=NumOfComponents
        parameters={ 
            "N":i,
            "MAMCS":MAMCS,
            "AdderSize":AdderSize,
            "IN_WIDTH":dpInwidthDic["dp{}".format(i)],
        }
    INPUT_REG_DEPTH = input('Enter INPUT_REG_DEPTH Default Value (empty for 1): ')
    #MULT_PIPE_DEPTH>=0
    MULT_PIPE_DEPTH = input('Enter MULT_PIPE_DEPTH Default Value (empty for 1): ')
    if not INPUT_REG_DEPTH:
        INPUT_REG_DEPTH=1
    else:
        INPUT_REG_DEPTH = int(INPUT_REG_DEPTH)
        if(INPUT_REG_DEPTH<0):
            INPUT_REG_DEPTH=0

    if not MULT_PIPE_DEPTH:
        MULT_PIPE_DEPTH=1
    else:
        MULT_PIPE_DEPTH = int(MULT_PIPE_DEPTH)
        if(MULT_PIPE_DEPTH<0):
            MULT_PIPE_DEPTH=0
    parameters["INPUT_REG_DEPTH"]=INPUT_REG_DEPTH
    parameters["MULT_PIPE_DEPTH"]=MULT_PIPE_DEPTH
    print(HRFlag)
    print(parameters)
    if(HRFlag):
        dpNamesDic["dp{}".format(i)]=dotProductModuleName=DotProduct_SI_AXI_Stream(parameters,Print_To_File,Files_Location,0,modules)
    else:
        dpNamesDic["dp{}".format(i)]=dotProductModuleName=DotProuct_noHR_AXIStream(parameters,Print_To_File,Files_Location,0,modules)
        print("no HR mode.")
    
print("module "+ModuleName,file=fileObject)
print("#(",file=fileObject)
print("parameter IN_DATA_LENGHT= {}, ".format(determinantIndataLenght),file=fileObject)
print("parameter OUT_DATA_LENGHT= {} ".format(dpOutdataLenghtDic["dp{}".format(N)]),file=fileObject)
print(")( \n",file=fileObject)
print("input aclk,".format(),file=fileObject)
print("input aresetn,".format(),file=fileObject)
print("input enable,".format(),file=fileObject)
print("input signed [IN_DATA_LENGHT-1:0]s_axi_data,".format(),file=fileObject)
print("input s_axi_valid,".format(),file=fileObject) 
print("input m_axi_ready,".format(),file=fileObject)
print("output signed [OUT_DATA_LENGHT-1:0]m_axi_data,".format(),file=fileObject)
print("output m_axi_valid,".format(),file=fileObject)
print("output s_axi_ready".format(),file=fileObject)
print(" );\n".format(),file=fileObject)
for i in range(3,N+1):
    print("reg [{}:{}]coefficientsShiftRegister_{};".format(determinantIndataLenght-1,0,i),file=fileObject)#for delayed coefficients from determinant.
dp2sReadyWire="dp2sReadyWire"
print("wire {};".format(dp2sReadyWire),file=fileObject)#dp2s connects to input s_axi_ ready
print("assign s_axi_ready= {};".format(dp2sReadyWire),file=fileObject)
dpNReadyWire="dpNReadyWire"#not used any more (previously used for making input s_axi_ready.)
#dp2sReadyWire="dp2sReadyWire"
#print("wire {};".format(dp2sReadyWire),file=fileObject)
#if(N>2):
#    dpNReadyWire="dpNReadyWire"
#    print("wire {};".format(dpNReadyWire),file=fileObject)
#if(N>2):
#    print("assign s_axi_ready= {}&&{};".format(dpNReadyWire,dp2sReadyWire),file=fileObject)
#else:
#    print("assign s_axi_ready= {};".format(dp2sReadyWire),file=fileObject)

outDataWireName="m_axi_data"
outReadyWireName="m_axi_ready"
outValidWireName="m_axi_valid"
determinant_NXN(N,N,RowMask,CollumnMask,-1,-1,dpInwidthDic,dpNamesDic,dpIndataLenghtDic,dpOutdataLenghtDic\
                                ,dpInstanceCount,fileObject,outDataWireName,outReadyWireName,outValidWireName,dp2sReadyWire\
                                ,dpNReadyWire,determinantElementWidth,determinantIndataLenght,dpInstancesDic,preventDuplicateDps)
statelenght=math.ceil(math.log2(N))+1
print("reg [{}:0]coefficientsShiftState;".format(statelenght-1),file=fileObject)
print("always@(posedge aclk)begin".format(),file=fileObject)
print(" if(!aresetn)begin".format(),file=fileObject)
for i in range(3,N+1):
    print("  coefficientsShiftRegister_{} <= 0;".format(i),file=fileObject)#for delayed coefficients from determinant.
print("  coefficientsShiftState <= 3;".format(i),file=fileObject)#for delayed coefficients from determinant.
print(" end".format(),file=fileObject)
print(" else begin".format(),file=fileObject)
print("  case(coefficientsShiftState)".format(),file=fileObject)
for i in range(3,N+2):
    print("   {}:begin".format(i),file=fileObject)
    if(i==3):
        print("    if(s_axi_ready && s_axi_valid)begin".format(),file=fileObject)
    else:
        print("    if(dp{}_0_s_axi_ready && dp{}_0_s_axi_valid)begin".format(i-1,i-1),file=fileObject)
    for j in range(3,N+1):
        if(j==3):
            print("     coefficientsShiftRegister_{} <= s_axi_data;".format(j),file=fileObject)
        else:
            print("     coefficientsShiftRegister_{} <= coefficientsShiftRegister_{};".format(j,j-1),file=fileObject)    
    if(i==N+1):
        print("     coefficientsShiftState<=3;".format(),file=fileObject)        
    else:
        print("     coefficientsShiftState<=coefficientsShiftState+1;".format(),file=fileObject)        
    print("    end".format(),file=fileObject)
    print("   end".format(),file=fileObject)
print("  endcase".format(),file=fileObject)
print(" end".format(),file=fileObject)
print("end".format(),file=fileObject)
print("endmodule".format(),file=fileObject)  
if Print_To_File>0:
    of.close()

In [25]:
print(dpNamesDic)
print(dpInwidthDic)
print(dpIndataLenghtDic)
print(dpOutdataLenghtDic)
print(dpInstanceCount)


{'dp2': 'DotProduct_2_SI_HRx2_C1_A0_IL_AXIStream', 'dp3': 'DotProduct_3_SI_HRx2_C2_A0_IL_AXIStream', 'dp4': 'DotProduct_4_SI_HRx2_C2_A0_IL_AXIStream', 'dp5': 'DotProduct_5_SI_HRx2_C3_A0_IL_AXIStream'}
{'dp2': 4, 'dp3': 9, 'dp4': 20, 'dp5': 42}
{'dp2': 16, 'dp3': 54, 'dp4': 160, 'dp5': 420}
{'dp2': 9, 'dp3': 20, 'dp4': 42, 'dp5': 87}
[0, 0, 60, 20, 5, 1]


In [16]:
import numpy as np
a = np.array([[5,3,0,5,6], [7,7,-1,5,-2],[1,3,3,5,6], [7,7,-1,1,-2],[1,3,7,5,6]])
print(a)
print (np.linalg.det(a))

[[ 5  3  0  5  6]
 [ 7  7 -1  5 -2]
 [ 1  3  3  5  6]
 [ 7  7 -1  1 -2]
 [ 1  3  7  5  6]]
3072.000000000001


In [7]:
x=[0, 1, 1, 0]
y=[0, 1, 1, 1]
b=''.join(map(str,y))+''.join(map(str,x))
print(b)
test="01100111"
print(test)
if(b==test):
    print("true")
else:
    print("false")

01110110
01100111
false


In [26]:
dic={
   "001":["hello1","hello1_1"] ,
   "010":"hello2" 
}
key="001"
if(key in dic):
    print(dic[key][0])
else:
    print("no exist")

hello1
